# 분석 전 기본세팅
> 분석 전 기본세팅, 필요코드 (불러오기, 내보내기, random forest, catboost, cross validation ..)

- toc: true 
- badges: true
- comments: true
- categories: [Python]
- image: images/

In [318]:
import pandas as pd
import numpy as np
df = pd.read_csv('http://bit.ly/ds-korean-idol')

# 정렬

## Index 별로 정렬

### 오름차순 정렬 df.sort_index()